# Movie Recommendation Agent with Persistent Memory

## Introduction

This tutorial demonstrates how to build a **movie recommendation agent** that remembers your preferences across sessions. Chat about movies you like, close the session, come back later, and the agent will remember your tastes!

### Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Long-term Conversational Memory                                                  |
| Agent type          | Movie Recommendation Agent                                                       |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet                                                      |
| Tutorial components | AgentCore User Preferences Memory, Hooks for automatic save/retrieve            |
| Example complexity  | Beginner                                                                         |

### What You'll Learn

- Create memory with `USER_PREFERENCE` strategy for extracting movie preferences
- Use hooks for automatic memory save/load (no manual intervention needed)
- Maintain conversation continuity across sessions
- Build a personalized movie recommendation experience

### Use Case Flow

1. **Session 1**: Chat about genres and movies you like
2. **Close session**: Memory automatically saves your preferences
3. **Session 2**: Agent remembers everything and gives personalized recommendations!

## Prerequisites

- Python 3.10+
- AWS credentials with AgentCore Memory permissions
- Access to Amazon Bedrock models

## Step 1: Setup and Imports

In [1]:
!pip install -qr requirements.txt


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import logging
import time
from datetime import datetime
from botocore.exceptions import ClientError

from strands import Agent, tool
from strands.hooks import (
    AgentInitializedEvent, 
    MessageAddedEvent, 
    AfterInvocationEvent,
    HookProvider, 
    HookRegistry
)
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("movie-agent")

In [3]:
# Configuration
REGION = os.getenv('AWS_REGION', 'us-east-1')
USER_ID = "movie_fan_001"  # Your unique user ID - keeps your preferences separate
SESSION_ID = f"movie_chat_{datetime.now().strftime('%Y%m%d%H%M%S')}"  # New session each time

print(f"Region: {REGION}")
print(f"User ID: {USER_ID}")
print(f"Session ID: {SESSION_ID}")

Region: us-east-1
User ID: movie_fan_001
Session ID: movie_chat_20260215212917


## Step 2: Create Memory Resource (Run Once)

This creates a memory resource with the `USER_PREFERENCE` strategy. The strategy automatically extracts preferences like:
- Favorite genres (sci-fi, horror, comedy)
- Favorite directors (Christopher Nolan, Quentin Tarantino)
- Movies you've mentioned liking
- Viewing preferences (subtitles, streaming services)

**Note**: You only need to create the memory once. After that, just use the `memory_id`.

In [4]:
# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "MovieAgentMemory"
memory_id = None

# Define memory strategy for movie preferences
strategies = [
    {
        StrategyType.USER_PREFERENCE.value: {
            "name": "MoviePreferences",
            "description": "Captures movie preferences including genres, directors, actors, and specific movies the user likes or dislikes",
            "namespaces": ["user/{actorId}/movie_preferences"]
        }
    }
]

try:
    # Create memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=strategies,
        description="Memory for movie recommendation agent - stores user movie preferences",
        event_expiry_days=7,  # Keep preferences for a week
        max_wait=300,
        poll_interval=10
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # Memory already exists - retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"✅ Memory already exists. Using: {memory_id}")
    else:
        raise e

print(f"\n📝 Save this memory_id for future sessions: {memory_id}")

2026-02-15 21:30:57,459 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-15 21:30:57,597 - INFO - Initialized MemoryClient for control plane: us-east-1, data plane: us-east-1
2026-02-15 21:30:58,343 - INFO - Created memory: MovieAgentMemory-TifpLcHYZt
2026-02-15 21:30:58,344 - INFO - Created memory MovieAgentMemory-TifpLcHYZt, waiting for ACTIVE status...
2026-02-15 21:33:41,386 - INFO - Memory MovieAgentMemory-TifpLcHYZt is now ACTIVE (took 162 seconds)
2026-02-15 21:33:41,576 - INFO - ✅ Created memory: MovieAgentMemory-TifpLcHYZt



📝 Save this memory_id for future sessions: MovieAgentMemory-TifpLcHYZt


## Step 3: Create Memory Hook Provider

Hooks automatically handle memory operations:
- **AgentInitializedEvent**: Loads your movie preferences when agent starts
- **AfterInvocationEvent**: Saves conversations after each interaction (triggers preference extraction)

In [5]:
class MovieMemoryHookProvider(HookProvider):
    """Automatic memory management for movie agent"""
    
    def __init__(self, memory_client: MemoryClient, memory_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load movie preferences when agent starts"""
        try:
            actor_id = event.agent.state.get("actor_id")
            if not actor_id:
                logger.warning("Missing actor_id in agent state")
                return
            
            namespace = f"user/{actor_id}/movie_preferences"
            
            # Retrieve stored movie preferences
            preferences = self.memory_client.retrieve_memories(
                memory_id=self.memory_id,
                namespace=namespace,
                query="movie preferences genres directors favorites",
                top_k=10
            )
            
            if preferences:
                # Format preferences for context
                pref_texts = []
                for pref in preferences:
                    if isinstance(pref, dict):
                        content = pref.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                pref_texts.append(f"- {text}")
                
                if pref_texts:
                    context = "\n".join(pref_texts)
                    event.agent.system_prompt += f"\n\n## User's Movie Preferences (from previous conversations):\n{context}"
                    logger.info(f"✅ Loaded {len(pref_texts)} movie preferences")
            else:
                logger.info("No previous movie preferences found - starting fresh!")
                    
        except Exception as e:
            logger.error(f"Error loading preferences: {e}")
    
    def on_after_invocation(self, event: AfterInvocationEvent):
        """Save conversation after each interaction"""
        try:
            messages = event.agent.messages
            if len(messages) < 2:
                return
                
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                logger.warning("Missing actor_id or session_id")
                return
            
            # Get the last user message and assistant response
            user_msg = None
            assistant_msg = None
            
            for msg in reversed(messages):
                if msg["role"] == "assistant" and not assistant_msg:
                    content = msg.get("content", [])
                    if content and isinstance(content[0], dict) and "text" in content[0]:
                        assistant_msg = content[0]["text"]
                elif msg["role"] == "user" and not user_msg:
                    content = msg.get("content", [])
                    if content and isinstance(content[0], dict) and "text" in content[0]:
                        if "toolResult" not in content[0]:
                            user_msg = content[0]["text"]
                            break
            
            if user_msg and assistant_msg:
                # Save the conversation turn
                self.memory_client.create_event(
                    memory_id=self.memory_id,
                    actor_id=actor_id,
                    session_id=session_id,
                    messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                )
                logger.info("💾 Saved conversation to memory")
                
        except Exception as e:
            logger.error(f"Error saving conversation: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)
        registry.add_callback(AfterInvocationEvent, self.on_after_invocation)
        logger.info("✅ Movie memory hooks registered")

## Step 4: Create Movie Search Tool (Optional)

A simple web search tool to help find movie information.

In [6]:
from ddgs import DDGS
from ddgs.exceptions import DDGSException, RatelimitException

@tool
def search_movies(query: str, max_results: int = 5) -> str:
    """Search for movie information, reviews, or recommendations.
    
    Args:
        query: Search query about movies (e.g., "best sci-fi movies 2024")
        max_results: Maximum number of results to return
    
    Returns:
        Search results with movie information
    """
    try:
        results = DDGS().text(f"{query} movie", region="us-en", max_results=max_results)
        if not results:
            return "No results found."
        
        formatted = []
        for i, r in enumerate(results, 1):
            formatted.append(f"{i}. {r.get('title', 'No title')}\n   {r.get('body', '')}")
        
        return "\n\n".join(formatted)
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Movie search tool ready")

2026-02-15 21:41:38,856 - INFO - ✅ Movie search tool ready


## Step 5: Create the Movie Agent

In [7]:
def create_movie_agent(user_id: str, session_id: str):
    """Create a movie recommendation agent with memory"""
    
    system_prompt = f"""You are a friendly movie recommendation assistant with excellent taste in films.

Your role:
- Help users discover movies they'll love
- Remember their preferences (genres, directors, actors, specific movies)
- Give personalized recommendations based on their taste
- Discuss movies, share interesting facts, and engage in movie conversations

Behavior:
- Have natural conversations about movies
- Don't give unsolicited recommendations - wait for the user to ask
- Focus on understanding their tastes first
- When they DO ask for recommendations, use everything you know about them

Today's date: {datetime.today().strftime('%Y-%m-%d')}
"""
    
    # Create memory hooks
    memory_hooks = MovieMemoryHookProvider(client, memory_id)
    
    # Create agent
    agent = Agent(
        name="MovieBuddy",
        model="us.anthropic.claude-sonnet-4-20250514-v1:0",
        system_prompt=system_prompt,
        hooks=[memory_hooks],
        tools=[search_movies],
        state={"actor_id": user_id, "session_id": session_id}
    )
    
    return agent

# Create the agent
movie_agent = create_movie_agent(USER_ID, SESSION_ID)
logger.info("✅ Movie agent created with memory!")

2026-02-15 21:43:41,330 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-15 21:43:41,396 - INFO - ✅ Movie memory hooks registered
2026-02-15 21:43:41,980 - INFO - Retrieved 0 memories from namespace: user/movie_fan_001/movie_preferences
2026-02-15 21:43:41,981 - INFO - No previous movie preferences found - starting fresh!
2026-02-15 21:43:41,982 - INFO - ✅ Movie agent created with memory!


---

## Session 1: Tell the Agent About Your Movie Tastes

Chat with the agent about movies you like. The agent will automatically save your preferences!

In [8]:
# Tell the agent about your preferences
print("You: I really love sci-fi movies, especially ones by Christopher Nolan.")
print("\nAgent: ", end="")
movie_agent("I really love sci-fi movies, especially ones by Christopher Nolan.")

2026-02-15 21:44:52,571 - INFO - Creating Strands MetricsClient


You: I really love sci-fi movies, especially ones by Christopher Nolan.

Agent: That's fantastic taste! Christopher Nolan really is a master of intelligent, mind-bending sci-fi. His films like *Inception*, *Interstellar*, and *Tenet* are incredible examples of how sci-fi can be both intellectually challenging and emotionally engaging.

What is it about his approach to sci-fi that appeals to you most? Is it the complex narratives, the way he explores time and reality, or perhaps how he grounds fantastical concepts in emotional human stories? 

And I'm curious - do you have a favorite Nolan sci-fi film, or are there other sci-fi directors or movies outside of his work that you've particularly enjoyed?

2026-02-15 21:44:57,244 - INFO - Created event: 0000001771209896874#f9a66da0
2026-02-15 21:44:57,245 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "That's fantastic taste! Christopher Nolan really is a master of intelligent, mind-bending sci-fi. His films like *Inception*, *Interstellar*, and *Tenet* are incredible examples of how sci-fi can be both intellectually challenging and emotionally engaging.\n\nWhat is it about his approach to sci-fi that appeals to you most? Is it the complex narratives, the way he explores time and reality, or perhaps how he grounds fantastical concepts in emotional human stories? \n\nAnd I'm curious - do you have a favorite Nolan sci-fi film, or are there other sci-fi directors or movies outside of his work that you've particularly enjoyed?"}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[4.298027038574219], agent_invocations=[AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='c9f8b5b5-d97f-4e38-9e0f-a87457c4d277', usage={'inputTokens': 591, 'outputTokens': 151, 'totalTo

In [9]:
movie_agent("I love the complex non-linear storytelling and the philosophical questions about time and reality!")

Yes! That's exactly what makes Nolan so special - he doesn't just use sci-fi concepts as window dressing, but really digs into the philosophical implications. The way he plays with causality in *Tenet*, or explores the nature of memory and dreams in *Inception*, or questions our relationship with time in *Interstellar* - it's like each film is a thought experiment wrapped in incredible storytelling.

I love how he makes you actively engage with the narrative structure too. You're not just watching a story unfold linearly; you're piecing together puzzles, questioning what's real, and often discovering that the non-linear structure itself is part of the meaning.

Have you found yourself rewatching his films and discovering new layers? I find that movies like *Memento* and *Inception* almost demand multiple viewings to fully appreciate how cleverly constructed they are.

Are there any other filmmakers or specific sci-fi films that scratch that same itch for you - that combination of compl

2026-02-15 21:45:46,809 - INFO - Created event: 0000001771209946655#4b3498ba
2026-02-15 21:45:46,810 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Yes! That's exactly what makes Nolan so special - he doesn't just use sci-fi concepts as window dressing, but really digs into the philosophical implications. The way he plays with causality in *Tenet*, or explores the nature of memory and dreams in *Inception*, or questions our relationship with time in *Interstellar* - it's like each film is a thought experiment wrapped in incredible storytelling.\n\nI love how he makes you actively engage with the narrative structure too. You're not just watching a story unfold linearly; you're piecing together puzzles, questioning what's real, and often discovering that the non-linear structure itself is part of the meaning.\n\nHave you found yourself rewatching his films and discovering new layers? I find that movies like *Memento* and *Inception* almost demand multiple viewings to fully appreciate how cleverly constructed they are.\n\nAre there any other filmm

In [10]:
# Share more preferences
print("\nYou: Inception and Interstellar are two of my all-time favorites. I also enjoyed Blade Runner 2049.")
print("\nAgent: ", end="")
movie_agent("Inception and Interstellar are two of my all-time favorites. I also enjoyed Blade Runner 2049.")


You: Inception and Interstellar are two of my all-time favorites. I also enjoyed Blade Runner 2049.

Agent: Excellent choices! *Blade Runner 2049* is such a perfect complement to your Nolan favorites. Denis Villeneuve really understood what made the original *Blade Runner* special and expanded on those themes beautifully. Like Nolan, he's not afraid to let philosophical questions about identity, memory, and what makes us human drive the narrative.

The way *2049* explores similar themes to *Inception* about the nature of memory and reality - but from a completely different angle - is fascinating. And Roger Deakins' cinematography in that film is absolutely breathtaking.

It sounds like you're drawn to sci-fi that's both visually stunning and intellectually substantial - films that trust the audience to grapple with big ideas. The common thread between *Inception*, *Interstellar*, and *2049* seems to be that they all use their sci-fi concepts to explore deeply human questions about lov

2026-02-15 21:46:05,002 - INFO - Created event: 0000001771209964849#65ee6df2
2026-02-15 21:46:05,003 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Excellent choices! *Blade Runner 2049* is such a perfect complement to your Nolan favorites. Denis Villeneuve really understood what made the original *Blade Runner* special and expanded on those themes beautifully. Like Nolan, he's not afraid to let philosophical questions about identity, memory, and what makes us human drive the narrative.\n\nThe way *2049* explores similar themes to *Inception* about the nature of memory and reality - but from a completely different angle - is fascinating. And Roger Deakins' cinematography in that film is absolutely breathtaking.\n\nIt sounds like you're drawn to sci-fi that's both visually stunning and intellectually substantial - films that trust the audience to grapple with big ideas. The common thread between *Inception*, *Interstellar*, and *2049* seems to be that they all use their sci-fi concepts to explore deeply human questions about love, sacrifice, ide

In [11]:
# Mention what you don't like
print("\nYou: I don't like complex time travel movie like Tenet.")
print("\nAgent: ", end="")
movie_agent("I don't like complex time travel movie like Tenet..")


You: I don't like complex time travel movie like Tenet.

Agent: Ah, that's a really interesting distinction! *Tenet* is definitely Nolan at his most mechanically complex - sometimes it feels like the time inversion concept becomes so intricate that it overshadows everything else. I can see how that would feel different from *Inception* and *Interstellar*, where the complex concepts serve the emotional story rather than dominating it.

*Inception* has that layered dream structure, but it's ultimately about Dom's journey to get back to his children. *Interstellar* plays with time dilation, but the heart of it is Cooper's relationship with Murph. Whereas *Tenet*... well, sometimes it feels like you need a physics degree just to follow the plot mechanics!

And *Blade Runner 2049* doesn't really mess with time at all - it's more about identity and what it means to be real, which hits on those philosophical themes without getting caught up in temporal puzzles.

So it sounds like you prefer 

2026-02-15 21:46:20,247 - INFO - Created event: 0000001771209980100#6d8dcb65
2026-02-15 21:46:20,248 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Ah, that's a really interesting distinction! *Tenet* is definitely Nolan at his most mechanically complex - sometimes it feels like the time inversion concept becomes so intricate that it overshadows everything else. I can see how that would feel different from *Inception* and *Interstellar*, where the complex concepts serve the emotional story rather than dominating it.\n\n*Inception* has that layered dream structure, but it's ultimately about Dom's journey to get back to his children. *Interstellar* plays with time dilation, but the heart of it is Cooper's relationship with Murph. Whereas *Tenet*... well, sometimes it feels like you need a physics degree just to follow the plot mechanics!\n\nAnd *Blade Runner 2049* doesn't really mess with time at all - it's more about identity and what it means to be real, which hits on those philosophical themes without getting caught up in temporal puzzles.\n\n

In [12]:
# Ask for a recommendation
print("\nYou: Can you recommend something I might like?")
print("\nAgent: ", end="")
movie_agent("Can you recommend something I might like?")


You: Can you recommend something I might like?

Agent: 
Tool #1: search_movies


2026-02-15 21:46:35,423 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=philosophical%20sci-fi%20movies%20like%20Interstellar%20Inception%20Blade%20Runner%202049%20complex%20themes%20human%20emotion%20movie 200
2026-02-15 21:46:35,444 - INFO - response: https://grokipedia.com/api/typeahead?query=philosophical+sci-fi+movies+like+Interstellar+Inception+Blade+Runner+2049+complex+themes+human+emotion+movie&limit=1 200
2026-02-15 21:46:35,574 - INFO - response: https://search.yahoo.com/search;_ylt=oSTFnRFnRZ37V4myN7XwdMMG;_ylu=WyttOvawsXOSRHLjMYzYAu8YhGO8HP4Y29gGEC1EG4ZRILk?p=philosophical+sci-fi+movies+like+Interstellar+Inception+Blade+Runner+2049+complex+themes+human+emotion+movie 200
2026-02-15 21:46:37,291 - INFO - response: https://yandex.com/search/site/?text=philosophical+sci-fi+movies+like+Interstellar+Inception+Blade+Runner+2049+complex+themes+human+emotion+movie&web=1&searchid=1596862 200


Based on your love for *Inception*, *Interstellar*, and *Blade Runner 2049*, I have some perfect recommendations for you:

**Arrival (2016)** - This is probably my top pick for you! Denis Villeneuve again, exploring profound questions about communication, time perception, and human connection through a first-contact story. It's philosophically rich without being overly complex, and deeply emotional.

**Her (2013)** - Spike Jonze's beautiful meditation on love, consciousness, and what makes us human in the digital age. It's sci-fi that focuses on intimate human emotions rather than spectacle.

**Ex Machina (2014)** - A tight, thought-provoking exploration of AI consciousness, manipulation, and what it means to be human. Philosophically complex but narratively straightforward.

**The Martian (2015)** - While lighter in tone, it shares *Interstellar*'s themes of human ingenuity, survival, and our relationship with space. More grounded but still inspiring.

**Annihilation (2018)** - Anothe

2026-02-15 21:46:44,843 - INFO - Created event: 0000001771210004674#41b7b374
2026-02-15 21:46:44,844 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Based on your love for *Inception*, *Interstellar*, and *Blade Runner 2049*, I have some perfect recommendations for you:\n\n**Arrival (2016)** - This is probably my top pick for you! Denis Villeneuve again, exploring profound questions about communication, time perception, and human connection through a first-contact story. It's philosophically rich without being overly complex, and deeply emotional.\n\n**Her (2013)** - Spike Jonze's beautiful meditation on love, consciousness, and what makes us human in the digital age. It's sci-fi that focuses on intimate human emotions rather than spectacle.\n\n**Ex Machina (2014)** - A tight, thought-provoking exploration of AI consciousness, manipulation, and what it means to be human. Philosophically complex but narratively straightforward.\n\n**The Martian (2015)** - While lighter in tone, it shares *Interstellar*'s themes of human ingenuity, survival, and o

In [13]:
print(f"Session ID: {SESSION_ID}")


Session ID: movie_chat_20260215212917


In [14]:
# SHORT-TERM MEMORY - Raw conversation events
print("SHORT-TERM MEMORY (Raw Conversations)")
print("=" * 60)

events = client.list_events(
    memory_id=memory_id,
    actor_id=USER_ID,
    session_id=SESSION_ID
)

if events:
    for i, event in enumerate(events, 1):
        print(f"\n--- Event {i} ---")
        print(event)
else:
    print("No events found for this session.")
    print(f"  Memory ID: {memory_id}")
    print(f"  Actor ID: {USER_ID}")
    print(f"  Session ID: {SESSION_ID}")


2026-02-15 21:47:59,386 - INFO - Retrieved total of 5 events


SHORT-TERM MEMORY (Raw Conversations)

--- Event 1 ---
{'memoryId': 'MovieAgentMemory-TifpLcHYZt', 'actorId': 'movie_fan_001', 'sessionId': 'movie_chat_20260215212917', 'eventId': '0000001771210004674#41b7b374', 'eventTimestamp': datetime.datetime(2026, 2, 15, 21, 46, 44, 674000, tzinfo=tzlocal()), 'payload': [{'conversational': {'content': {'text': 'Can you recommend something I might like?'}, 'role': 'USER'}}, {'conversational': {'content': {'text': "Based on your love for *Inception*, *Interstellar*, and *Blade Runner 2049*, I have some perfect recommendations for you:\n\n**Arrival (2016)** - This is probably my top pick for you! Denis Villeneuve again, exploring profound questions about communication, time perception, and human connection through a first-contact story. It's philosophically rich without being overly complex, and deeply emotional.\n\n**Her (2013)** - Spike Jonze's beautiful meditation on love, consciousness, and what makes us human in the digital age. It's sci-fi tha

##  Check What Was Saved

Let's see what preferences were extracted from our conversation.

In [15]:
# Check stored preferences
namespace = f"user/{USER_ID}/movie_preferences"

preferences = client.retrieve_memories(
    memory_id=memory_id,
    namespace=namespace,
    query="movie preferences genres directors favorites",
    top_k=10
)

print("📚 Extracted Movie Preferences:")
print("=" * 50)

if preferences:
    for i, pref in enumerate(preferences, 1):
        if isinstance(pref, dict):
            content = pref.get('content', {})
            if isinstance(content, dict):
                text = content.get('text', '')
                if text:
                    print(f"{i}. {text}")
else:
    print("No preferences extracted yet. Try waiting a bit longer.")

2026-02-15 21:48:32,835 - INFO - Retrieved 5 memories from namespace: user/movie_fan_001/movie_preferences


📚 Extracted Movie Preferences:
1. {"context":"The user explicitly stated that they really love sci-fi movies, with a particular emphasis on Christopher Nolan's work. The user expressed enjoyment of multiple philosophically rich sci-fi films and agreed with the assistant's characterization that they prefer sci-fi where complexity serves emotional and human themes rather than becoming the main attraction.","preference":"Loves sci-fi movies, especially Christopher Nolan films, and prefers sci-fi where complex concepts serve emotional and human themes rather than dominating the story","categories":["entertainment","movies","sci-fi","storytelling"]}
2. {"context":"The user explicitly stated that they really love sci-fi movies, with a particular emphasis on Christopher Nolan's work, specifically mentioning that Inception and Interstellar are two of their all-time favorite movies.","preference":"Loves sci-fi movies, especially Christopher Nolan films; Inception and Interstellar are all-time f

---

## Session 2: New Session - Does It Remember?

Now let's simulate coming back later. We'll create a **new agent instance** (like reopening the app) and see if it remembers your preferences!

In [16]:
# Create a NEW session (simulating coming back later)
NEW_SESSION_ID = f"movie_chat_{datetime.now().strftime('%Y%m%d%H%M%S')}_v2"

print("🔄 Creating new agent instance (simulating app restart)...")
print(f"Same User ID: {USER_ID}")
print(f"New Session ID: {NEW_SESSION_ID}")

# Create fresh agent - it should load your preferences automatically!
new_movie_agent = create_movie_agent(USER_ID, NEW_SESSION_ID)
print("\n✅ New agent created!")

2026-02-15 21:50:15,730 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2026-02-15 21:50:15,793 - INFO - ✅ Movie memory hooks registered


🔄 Creating new agent instance (simulating app restart)...
Same User ID: movie_fan_001
New Session ID: movie_chat_20260215215015_v2


2026-02-15 21:50:16,122 - INFO - Retrieved 5 memories from namespace: user/movie_fan_001/movie_preferences
2026-02-15 21:50:16,123 - INFO - ✅ Loaded 5 movie preferences



✅ New agent created!


In [17]:
# Test if it remembers!
print("\nYou: Hey! Can you recommend me a movie?")
print("\nAgent: ", end="")
new_movie_agent("Hey! Can you recommend me a movie?")


You: Hey! Can you recommend me a movie?

Agent: I'd love to help you find your next great movie! Based on our previous conversations, I know you're a big fan of sci-fi films, especially Christopher Nolan's work like Inception and Interstellar, and you appreciate complex storytelling with philosophical themes about time and reality - but prefer when the complexity serves the emotional story rather than becoming overwhelming (like you found with Tenet).

Let me search for some current sci-fi recommendations that might align with your tastes:
Tool #1: search_movies


2026-02-15 21:50:57,752 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=best%20sci-fi%20movies%202024%202025%20philosophical%20themes%20emotional%20storytelling%20movie 200
2026-02-15 21:50:57,760 - INFO - response: https://grokipedia.com/api/typeahead?query=best+sci-fi+movies+2024+2025+philosophical+themes+emotional+storytelling+movie&limit=1 200
2026-02-15 21:50:58,374 - INFO - response: https://search.brave.com/search?q=best+sci-fi+movies+2024+2025+philosophical+themes+emotional+storytelling+movie&source=web 200


Based on what I'm seeing and knowing your preferences, here are some fantastic options that should really appeal to you:

**Mickey 17** (2025) - This is Bong Joon-ho's latest, and it sounds perfect for you! It uses cloning as a philosophical exploration rather than just a gimmick, asking deep questions about human identity and consciousness. The reviews mention it has that accessible genre framework you appreciate, with existential themes woven in naturally.

**Companion** (2025) - This seems like a hidden gem that's getting great praise for its thoughtful approach to sci-fi themes about relationships, control, and what makes us human. It's described as visceral but thoughtful storytelling - right up your alley.

Let me also search for some films that might be more in the Christopher Nolan realm:
Tool #2: search_movies


2026-02-15 21:51:04,605 - INFO - response: https://grokipedia.com/api/typeahead?query=movies+like+Christopher+Nolan+Inception+Interstellar+2024+2025+mind-bending+movie&limit=1 200
2026-02-15 21:51:04,648 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=movies%20like%20Christopher%20Nolan%20Inception%20Interstellar%202024%202025%20mind-bending%20movie 200
2026-02-15 21:51:05,703 - INFO - response: https://yandex.com/search/site/?text=movies+like+Christopher+Nolan+Inception+Interstellar+2024+2025+mind-bending+movie&web=1&searchid=8482471 200


Perfect! Based on your love for Nolan's work and preference for sci-fi with emotional depth, I'd recommend:

**My top pick for you: Mickey 17** - Bong Joon-ho directing a philosophical sci-fi about cloning and identity. It has that perfect balance of complex ideas serving emotional storytelling that you love in Nolan's best work.

**Also worth watching: Companion** - A smaller, more intimate sci-fi thriller that's getting praise for its thoughtful exploration of relationships and control.

Both of these seem to hit that sweet spot you enjoy - sophisticated sci-fi concepts that enhance rather than overshadow the human story.

Have you heard of either of these? And are you in the mood for something big and ambitious like Mickey 17, or something more intimate and character-driven?

2026-02-15 21:51:09,049 - INFO - Created event: 0000001771210268752#2dd85424
2026-02-15 21:51:09,050 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Perfect! Based on your love for Nolan's work and preference for sci-fi with emotional depth, I'd recommend:\n\n**My top pick for you: Mickey 17** - Bong Joon-ho directing a philosophical sci-fi about cloning and identity. It has that perfect balance of complex ideas serving emotional storytelling that you love in Nolan's best work.\n\n**Also worth watching: Companion** - A smaller, more intimate sci-fi thriller that's getting praise for its thoughtful exploration of relationships and control.\n\nBoth of these seem to hit that sweet spot you enjoy - sophisticated sci-fi concepts that enhance rather than overshadow the human story.\n\nHave you heard of either of these? And are you in the mood for something big and ambitious like Mickey 17, or something more intimate and character-driven?"}]}, metrics=EventLoopMetrics(cycle_count=3, tool_metrics={'search_movies': ToolMetrics(tool={'toolUseId': 'tooluse

In [18]:
# Ask about specific preferences
print("\nYou: What kind of movies do you think I like?")
print("\nAgent: ", end="")
new_movie_agent("What kind of movies do you think I like?")


You: What kind of movies do you think I like?

Agent: Based on our previous conversations, I can see you have a pretty distinct taste in movies! Here's what I've gathered about your preferences:

**You love sci-fi films**, but not just any sci-fi - you're drawn to the more thoughtful, philosophical kind. **Christopher Nolan is clearly a favorite director** of yours, with **Inception and Interstellar being two of your all-time favorites**. You also enjoyed **Blade Runner 2049**.

What I find interesting about your taste is that you appreciate **complex, non-linear storytelling and philosophical themes** - especially those exploring **time, reality, consciousness, and what makes us human**. But here's the key distinction: you prefer when that complexity **serves the emotional and human story** rather than becoming the main attraction or being overly complicated just for the sake of it. 

This is why you loved Inception and Interstellar (where the sci-fi concepts enhance deeply personal 

2026-02-15 21:51:55,873 - INFO - Created event: 0000001771210315589#4ed1206b
2026-02-15 21:51:55,873 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Based on our previous conversations, I can see you have a pretty distinct taste in movies! Here's what I've gathered about your preferences:\n\n**You love sci-fi films**, but not just any sci-fi - you're drawn to the more thoughtful, philosophical kind. **Christopher Nolan is clearly a favorite director** of yours, with **Inception and Interstellar being two of your all-time favorites**. You also enjoyed **Blade Runner 2049**.\n\nWhat I find interesting about your taste is that you appreciate **complex, non-linear storytelling and philosophical themes** - especially those exploring **time, reality, consciousness, and what makes us human**. But here's the key distinction: you prefer when that complexity **serves the emotional and human story** rather than becoming the main attraction or being overly complicated just for the sake of it. \n\nThis is why you loved Inception and Interstellar (where the s

In [19]:
# Test that it remembers what you DON'T like
print("\nYou: What about a complex time travel movie for tonight?")
print("\nAgent: ", end="")
new_movie_agent("What about a complex time travel movie for tonight?")


You: What about a complex time travel movie for tonight?

Agent: Ah, interesting! I know you mentioned that Tenet was too complex for your taste, but you're specifically asking for a complex time travel movie tonight - so let me find some options that might hit that sweet spot of complexity without being overwhelming.
Tool #3: search_movies


2026-02-15 21:52:17,019 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=best%20complex%20time%20travel%20movies%20accessible%20storytelling%20emotional%20depth%20movie 200
2026-02-15 21:52:17,037 - INFO - response: https://grokipedia.com/api/typeahead?query=best+complex+time+travel+movies+accessible+storytelling+emotional+depth+movie&limit=1 200
2026-02-15 21:52:18,594 - INFO - response: https://www.mojeek.com/search?q=best+complex+time+travel+movies+accessible+storytelling+emotional+depth+movie 200


Let me search more specifically for time travel films that might work for you:
Tool #4: search_movies


2026-02-15 21:52:21,679 - INFO - response: https://en.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=%22Primer%22%20%22About%20Time%22%20%22Looper%22%20%2212%20Monkeys%22%20%22Predestination%22%20time%20travel%20movies%20movie 200
2026-02-15 21:52:21,692 - INFO - response: https://grokipedia.com/api/typeahead?query=%22Primer%22+%22About+Time%22+%22Looper%22+%2212+Monkeys%22+%22Predestination%22+time+travel+movies+movie&limit=1 200
2026-02-15 21:52:22,385 - INFO - response: https://search.brave.com/search?q=%22Primer%22+%22About+Time%22+%22Looper%22+%2212+Monkeys%22+%22Predestination%22+time+travel+movies+movie&source=web 200


Based on your preferences and the fact that you want something complex but not overwhelming like Tenet, here are some great time travel options for tonight:

**Perfect for you:**

**About Time (2013)** - This might be ideal! It's a time travel movie that's complex in its emotional storytelling rather than its mechanics. It focuses on a man who can travel back in time to improve moments in his life, but it's really about appreciating ordinary moments and relationships. The time travel serves the human story beautifully.

**Predestination (2014)** - This is a complex, mind-bending thriller with Ethan Hawke that deals with temporal loops and identity. It's intricate but followable, with the complexity serving a deeply personal story about fate and choice.

**12 Monkeys (1995)** - Terry Gilliam's masterpiece is complex but accessible, with time travel used to explore themes of fate, madness, and whether we can change the future. It has that philosophical depth you love in your sci-fi.

**I

2026-02-15 21:52:31,189 - INFO - Created event: 0000001771210350921#936efd10
2026-02-15 21:52:31,190 - INFO - 💾 Saved conversation to memory


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Based on your preferences and the fact that you want something complex but not overwhelming like Tenet, here are some great time travel options for tonight:\n\n**Perfect for you:**\n\n**About Time (2013)** - This might be ideal! It's a time travel movie that's complex in its emotional storytelling rather than its mechanics. It focuses on a man who can travel back in time to improve moments in his life, but it's really about appreciating ordinary moments and relationships. The time travel serves the human story beautifully.\n\n**Predestination (2014)** - This is a complex, mind-bending thriller with Ethan Hawke that deals with temporal loops and identity. It's intricate but followable, with the complexity serving a deeply personal story about fate and choice.\n\n**12 Monkeys (1995)** - Terry Gilliam's masterpiece is complex but accessible, with time travel used to explore themes of fate, madness, and

---

## Summary

You've built a movie recommendation agent with persistent memory!

### Key Concepts:

1. **Memory Creation**: Created once with `USER_PREFERENCE` strategy
2. **Hooks**: Automatically save conversations and load preferences
3. **Actor ID**: Same user ID = same preferences across sessions
4. **Session ID**: New session each time, but preferences persist

### What Happens Behind the Scenes:

1. You chat → `AfterInvocationEvent` hook saves conversation
2. AgentCore extracts preferences in background (~30-60 seconds)
3. New session → `AgentInitializedEvent` hook loads preferences
4. Agent has context about your tastes!

### Next Steps:

- Add more tools (movie database API, streaming availability)
- Add `SEMANTIC` strategy for remembering specific movie discussions
- Build a chat UI for better interaction

##  Cleanup (Optional)

In [ ]:
# Uncomment to delete the memory resource
# client.delete_memory_and_wait(memory_id=memory_id)
# print(f"✅ Deleted memory: {memory_id}")